In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv

In [2]:
def biggest_contour(contours,image_size):
    biggest = None
    max_area = 0
    biggest_n=0
    approx_contour=None
    for n,i in enumerate(contours):
            area = cv.contourArea(i)
         
            
            if area > image_size/10:
                    peri = cv.arcLength(i,True)
                    approx = cv.approxPolyDP(i,0.02*peri,True)
                    if area > max_area and len(approx)==4:
                            biggest = approx
                            max_area = area
                            biggest_n=n
                            approx_contour=approx
                            
                                                   
    return biggest_n,approx_contour

In [3]:
def order_points(pts):
    pts=pts.reshape(4,2)
    rect = np.zeros((4, 2), dtype = "float32")

    s = pts.sum(axis = 1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]
    
    diff = np.diff(pts, axis = 1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]

  
    return rect

In [4]:
def four_point_transform(image, pts):
    
    rect = order_points(pts)
    (tl, tr, br, bl) = rect


    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))
   

    
    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))

    dst = np.array([
        [0, 0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
        [0, maxHeight - 1]], dtype = "float32")

    M = cv.getPerspectiveTransform(rect, dst)
    warped = cv.warpPerspective(image, M, (maxWidth, maxHeight))

    
    return warped


In [5]:
def nesnetespiti(img):
    kernel = np.ones((5,5))
    image=img.copy()  
    height, width, channels = image.shape
    img_gray=cv.cvtColor(image,cv.COLOR_BGR2GRAY)
    image_size=img_gray.size

    
    cv.namedWindow("Gri", cv.WINDOW_AUTOSIZE) 
    temp = cv.resize(img_gray, (540, 960))   
    cv.imshow('Gri',temp)
    cv.waitKey(0)

    
    cv.namedWindow("Bulaniklastirilmis Goruntu", cv.WINDOW_AUTOSIZE) 
    blur = cv.blur(img_gray,(5,5),0)
    temp = cv.resize(blur, (540, 960))   
    cv.imshow('Bulaniklastirilmis Gorutunu',temp)
    cv.waitKey(0)
    cv.destroyAllWindows()

    
    cv.namedWindow("Esiklenmis Goruntu", cv.WINDOW_AUTOSIZE) 
    th3 = cv.adaptiveThreshold(blur,255,cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY,11,2)
    temp = cv.resize(th3, (540, 960))
    cv.imshow('Esiklenmis Goruntu',temp)
    cv.waitKey(0)
    cv.destroyAllWindows()

    
    
    cv.namedWindow("Denoising", cv.WINDOW_AUTOSIZE) 
    denoising = cv.fastNlMeansDenoising(th3, 11, 31, 9)
    temp = cv.resize(denoising, (540, 960))
    cv.imshow('Denoising',temp)
    cv.waitKey(0)
    cv.destroyAllWindows()
  
    

  
    cv.namedWindow("erosion", cv.WINDOW_AUTOSIZE) 
    erosion = cv.erode(denoising,kernel,iterations = 1)
    temp = cv.resize(erosion, (540, 960))
    cv.imshow('erosion',temp)
    cv.waitKey(0)
    cv.destroyAllWindows()
   
    
    cv.namedWindow("closing", cv.WINDOW_AUTOSIZE) 
    closing = cv.morphologyEx(erosion, cv.MORPH_CLOSE, kernel)
    temp = cv.resize(closing, (540, 960))
    cv.imshow('closing',temp)
    cv.waitKey(0)
    cv.destroyAllWindows()
  
    
    cv.namedWindow("Canny Edge Detection", cv.WINDOW_AUTOSIZE) 
    edges = cv.Canny(closing,50,150,apertureSize = 7)
    temp = cv.resize(edges, (540, 960))
    cv.imshow('Canny Edge Detection',temp)
    cv.waitKey(0)
    cv.destroyAllWindows()
  
   
    contours, hierarchy = cv.findContours(edges, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    simplified_contours = []


    for cnt in contours:
        hull = cv.convexHull(cnt)
        simplified_contours.append(cv.approxPolyDP(hull,
                                0.001*cv.arcLength(hull,True),True))
    simplified_contours = np.array(simplified_contours)
    biggest_n,approx_contour = biggest_contour(simplified_contours,image_size)

    threshold = cv.drawContours(image, simplified_contours ,biggest_n, (0,255,0), 1)
   
    dst = 0
    if approx_contour is not None and len(approx_contour)==4:
        approx_contour=np.float32(approx_contour)
        dst=four_point_transform(threshold,approx_contour)
    
    cropped = dst
    
    cv.namedWindow("Kirpilmis Goruntu", cv.WINDOW_AUTOSIZE)
    temp = cv.resize(cropped, (540, 960))
    cv.imshow("Kirpilmis Goruntu",temp)
    cv.waitKey(0)
    cv.destroyAllWindows()
    

    return cropped

In [6]:
def iyilestirme(img):
    filter = np.array([[-1,-1,-1],[-1,9,-1],[-1,-1,-1]])
    sharpened = cv.filter2D(img,-1,filter)
    cv.namedWindow("Keskinlestirilmis Goruntu", cv.WINDOW_AUTOSIZE)
    temp = cv.resize(sharpened, (540, 960))
    cv.imshow("Keskinlestirilmis Goruntu",temp)
    cv.waitKey(0)
    cv.destroyAllWindows()

    return sharpened
    

In [7]:
def scanner(img):#iki ana fonksiyon kullanılak tek fonksiyon kullanıldı.
    warped = nesnetespiti(img)
    sharpened=iyilestirme(warped)
    return sharpened

In [8]:
cv.namedWindow("input", cv.WINDOW_AUTOSIZE)      
input_img  =  cv . imread ('test1.jpeg' )   
input_img2 = cv.resize(input_img, (540, 960))                    
cv.imshow("input", input_img2)                            
cv.waitKey(0)               

output_img = scanner(input_img)
cv.imwrite('test1_out.jpeg',output_img)

True